<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##前提ファイルの読み込み (**とりあえず押して！**)  { display-mode: "form"}

from IPython.display import display, Math
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
from scipy.stats import norm

os.makedirs("data", exist_ok=True)
os.makedirs("result", exist_ok=True)

# 出力をクリア
from IPython.display import clear_output
clear_output()

## **S/N 計算**

In [ ]:
#@title ファイルを確認 { display-mode: "form"}
#@markdown ## **`data`フォルダーにデータを入れる**
#@markdown ##1. 実行ボタンを押してファイルを確認
files = os.listdir("data")
MRIs = [f for f in files if os.path.splitext(f)[1] == '.csv']
MRIs = np.sort(MRIs)

print(f"ファイル数: {len(MRIs)}")
print(f"1: {MRIs[0]} ")
if len(MRIs) > 1:
  print(f"2: {MRIs[1]} ")
else:
  print("......")
if len(MRIs) > 2:
  print(f"3: {MRIs[2]} ")
  print("......")
else:
  print("......")


In [ ]:
#@title S/N計算 { display-mode: "form"}
#@markdown ##2. 実行
#@markdown ##3. `result/`に`yy-mm-dd.csv`の結果が保存

def read_mpi(file, unit="V"):
  dat = pd.read_csv(file, header=None)
  dat = dat.T
  n = len(dat)
  Harmonics = pd.DataFrame(np.zeros([int(n/3), 3]) , columns=["base", "1h", "3h"])
  Harmonics["base"] = dat.iloc[3*np.arange(n/3)].values
  Harmonics["1h"] = dat.iloc[3*np.arange(n/3)+1].values
  Harmonics["3h"] = dat.iloc[3*np.arange(n/3)+2].values
  Harmonics = Harmonics.astype(float)
  if unit == "mV":
    Harmonics = Harmonics*1000
  return Harmonics

def snr_mpi(df):
  table = pd.DataFrame(np.zeros([1, 4]), columns=["1h", "1h_sd", "3h", "3h_sd"])
  table["1h"] = df["1h"].mean()
  table["1h_sd"] = df["1h"].std()
  table["3h"] = df["3h"].mean()
  table["3h_sd"] = df["3h"].std()
  return table

def table_mpi(files):
  n = len(files)
  mtable = pd.DataFrame(np.zeros([n, 5]), columns=["sample", "1h", "1h_sd", "3h", "3h_sd"])
  mtable["sample"] = [f.replace(".csv", "").replace("data/", "") for f in files]
  for i, file in enumerate(files):
    df = read_mpi(file, unit="mV")
    this_table = snr_mpi(df)
    mtable.iloc[i, :]["sample"] = file.replace(".csv", "").replace("data/", "")
    mtable.iloc[i, 1:5] = this_table
  return mtable

pathes = [os.path.join("data", f) for f in MRIs]
result_mpi = table_mpi(pathes)

print("\n======Result======")
print("n = {}".format(len(pathes)))
display(result_mpi)

today = pd.to_datetime("today").strftime("%Y-%m-%d")
save_path = os.path.join("result", f"{today}.csv")
result_mpi.to_csv(save_path)
print(f"\nSave: {save_path}に保存しました")

## **統計処理**

In [ ]:
#@title ファイルを確認 { display-mode: "form"}
#@markdown ## **`data`フォルダーにデータを入れる**
#@markdown ##1. 実行ボタンを押してファイルを確認
#@markdown ##2. 解析するファイルを選択 (実行ボタンは押さない)
files = os.listdir("data")
MRIs = [f for f in files if os.path.splitext(f)[1] == '.csv']
MRIs = np.sort(MRIs)

print(f"ファイル数: {len(MRIs)}")
print(f"1: {MRIs[0]} ")
if len(MRIs) > 1:
  print(f"2: {MRIs[1]} ")
else:
  print("......")
if len(MRIs) > 2:
  print(f"3: {MRIs[2]} ")
  print("......")
else:
  print("......")


In [ ]:
#@title サンプルを選択 { display-mode: "form", run: "auto" }
#@markdown ##共通の文字を入力
key_word = "Acetone" # @param {"type":"string"}


target_samples = [f for f in files if key_word in f]
target_samples = sorted(target_samples)

print(f"データは {len(target_samples)}個 見つかりました")
for i, target_sample in enumerate(target_samples):
  print(f"{i+1}: {target_sample}")
target_pathes = [os.path.join("data", target_sample) for target_sample in target_samples]

In [ ]:
#@title 統計処理 { display-mode: "form"}
#@markdown ##サンプル質量(mg)を入力
mass = 100 # @param {"type":"number"}
mass_g = mass / 1000
mass_kg = mass_g / 1000

#@markdown ##シグナルの質量単位を入力
signal_unit = "g" # @param ["mg","g","kg"]

if signal_unit == "g":
  signal_mass = mass_g
elif signal_unit == "kg":
  signal_mass = mass_kg
else:
  signal_mass = mass

#@markdown ##信頼区間(%)を入力
#@markdown ###※0だと標準誤差(SE)になります
CI = 0 # @param {"type":"number"}

def SE(nums, pcent=0):
  sample_size = len(nums)
  std_dev = np.std(nums, ddof=1)
  std_err = std_dev / np.sqrt(sample_size)
  if pcent == 0:
    return std_err
  else:
    if pcent > 100:
      pcent = 99.9
    elif pcent < 0:
      pcent = 1
    z_score = norm.ppf(1 - (1 - pcent/100) / 2)
    return std_err * z_score

def read_mpi(file, unit="V"):
  dat = pd.read_csv(file, header=None)
  dat = dat.T
  n = len(dat)
  Harmonics = pd.DataFrame(np.zeros([int(n/3), 3]) , columns=["base", "1h", "3h"])
  Harmonics["base"] = dat.iloc[3*np.arange(n/3)].values
  Harmonics["1h"] = dat.iloc[3*np.arange(n/3)+1].values
  Harmonics["3h"] = dat.iloc[3*np.arange(n/3)+2].values
  Harmonics = Harmonics.astype(float)
  if unit == "mV":
    Harmonics = Harmonics*1000
  return Harmonics

def table_mpi(files):
  n = len(files)
  mtable = pd.DataFrame(np.zeros([n, 4]), columns=["sample", "1h", "3h", "ratio"])
  mtable["sample"] = [f.replace(".csv", "").replace("data/", "") for f in files]
  for i, file in enumerate(files):
    dat = read_mpi(file, unit="mV")
    mtable.iloc[i, 1] = dat.mean()["1h"]
    mtable.iloc[i, 2] = dat.mean()["3h"]
    mtable.iloc[i, 3] = mtable.iloc[i, 2] / mtable.iloc[i, 1]
  return mtable

def stat_mpi(files, ci=0):
  if len(files) < 2:
    print("Too small the number of files!!")
    return None
  n = len(files)
  mtable = table_mpi(files)
  stable = pd.DataFrame(np.zeros([1, 6]),
                        columns=["1h", "d_1h", "3h", "d_3h", "ratio", "d_ratio"])
  stable["1h"] = mtable["1h"].mean()
  stable["3h"] = mtable["3h"].mean()
  stable["ratio"] = mtable["ratio"].mean()
  stable["d_1h"] = SE(mtable["1h"], pcent=ci)
  stable["d_3h"] = SE(mtable["3h"], pcent=ci)
  stable["d_ratio"] = SE(mtable["ratio"], pcent=ci)
  return stable

result_mpi = stat_mpi(target_pathes, ci=CI)
result_mpi["signal"] = result_mpi["ratio"] / signal_mass
result_mpi["d_signal"] = result_mpi["d_ratio"] / signal_mass

print("======Condition======")
print("n = {}".format(len(target_pathes)))
print("Confidence interval = {}%".format(CI))
print("Sample mass = {} mg".format(mass))
print("Signal unit = {}^-1".format(signal_unit))
print("\n======Result======")
# display(result_mpi)
display(Math(r"\text{{1st Harmonic}} = {:.4g} \pm {:.2g} \,\text{{mV}}".format(result_mpi["1h"][0], result_mpi['d_1h'][0])))
display(Math(r"\text{{3rd Harmonic}} = {:.3g} \pm {:.1g} \,\text{{mV}}".format(result_mpi["3h"][0], result_mpi['d_3h'][0])))
display(Math(r"\text{{Ratio}} = ( {:.3g} \pm {:.2g} ) \times 10^{{-3}}".format(1000*result_mpi["ratio"][0], 1000*result_mpi['d_ratio'][0])))
display(Math(r"\text{{Signal}} = {:.3g} \pm {:.2g} \,\text{{{}}}^{{-1}}".format(result_mpi["signal"][0], result_mpi['d_signal'][0], signal_unit)))

save_path = os.path.join("result", f"{key_word}.csv")
result_mpi.to_csv(save_path)
print(f"\nSave: {save_path}に保存しました")

In [ ]:
#@title 結果まとめ表を作成

def read_result(file, name="0"):
  dat = pd.read_csv(file, index_col=0)
  dat.index = [name]
  return dat

result_pathes = os.listdir("result")
result_pathes = [os.path.join("result", f) for f in result_pathes]
result_pathes = sorted(result_pathes)
result_samples = [f.replace(".csv", "").replace("result/", "") for f in result_pathes]

results = None
for i, result_path in enumerate(result_pathes):
  result = read_result(result_path, name=result_samples[i])
  if results is None:
    results = result
  else:
    results = pd.concat([results, result])

display(results)
save_path = os.path.join("result", "Summary.csv")
results.to_csv(save_path)
print(f"\nSave: {save_path}に保存しました")

In [ ]:
#@title `result.zip`を作成
!zip -r result.zip result/